# Coursera Week 3 Submission Part-3 (Continued after Part-1 and Part-2)
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown in the coursera website.

I did mention the steps below as to how I proceeded to create the final pandas Dataframe

### PART-1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [1]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files
import requests
from pandas.io.json import json_normalize

import requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

# I did not use Beatiful Soup for this as I felt this would be easier and faster
required_cols = ['Postcode', 'Borough','Neighbourhood']

# Note I converted the boolean output of array_equal function to String as it 
# wasn't working as expected when used in the if condition as expected
for table in tables:
    if(str(np.array_equal(np.array(table.columns),np.array(required_cols)))=="True"):
        pstl_data_df = pd.DataFrame(table)    
    break
print("Shape of Dataframe is - ",pstl_data_df.shape)
pstl_data_df.head()

Shape of Dataframe is -  (288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Step 2: Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
#Filtering out Boroughs which are Not assigned
pstl_data_df = pstl_data_df[pstl_data_df.Borough!="Not assigned"]
print("Shape of Dataframe is - ",pstl_data_df.shape)
pstl_data_df.head()

Shape of Dataframe is -  (211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Step 3: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [4]:
pstl_data_df['new_nghbr'] = np.where(pstl_data_df['Neighbourhood']=='Not assigned',pstl_data_df['Borough'],pstl_data_df['Neighbourhood'])
pstl_data_df.head(10)

,Postcode,Borough,Neighbourhood,new_nghbr
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,Harbourfront,Harbourfront
5,M5A,Downtown Toronto,Regent Park,Regent Park
6,M6A,North York,Lawrence Heights,Lawrence Heights
7,M6A,North York,Lawrence Manor,Lawrence Manor
8,M7A,Queen's Park,Not assigned,Queen's Park
10,M9A,Etobicoke,Islington Avenue,Islington Avenue
11,M1B,Scarborough,Rouge,Rouge
12,M1B,Scarborough,Malvern,Malvern


### Step 4: More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

Used the "new_nghbr" column to get a grouped column, as in wherever the same Postal Code and the same Borough are present we have to club the Neighbourhood column using the functions groupby and apply

In [5]:
can_postal_cd_df = pd.DataFrame(pstl_data_df.groupby(['Postcode','Borough'])['new_nghbr'].apply((', '.join)).reset_index())

# Renaming Columns to the same one in the image given:
can_postal_cd_df = can_postal_cd_df.rename(columns = {"Postcode": "PostalCode","new_nghbr":"Neighborhood"})

In [6]:
print(can_postal_cd_df.head())
print("\n The Final Shape of the dataframe is  - ",can_postal_cd_df.shape)

  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Rouge, Malvern
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

 The Final Shape of the dataframe is  -  (103, 3)


### As We see above the shape of the new data frame created is (103,3)

### PART-1 Ends Here

## PART-2 Now we proceed to add the Lat Long data to the existing dataframe

### Installing Geocoder

In [7]:
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import geocoder # import geocoder
print("Geo Coder imported!")

Solving environment: done

# All requested packages already installed.

Installation Done!
Geo Coder imported!


Creating a function to get the Lat Long data from the Postal Code

In [8]:
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

Adding the Latitude and Longitude columns to the Pandas DataFrame

In [9]:
can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'] = zip(*can_postal_cd_df['PostalCode'].apply(get_geocoder))
can_postal_cd_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [10]:
print("Shape of the dataframe is - ",can_postal_cd_df.shape)

Shape of the dataframe is -  (103, 5)


## Map of Toronto

Using geolocator for Mapping Toronto as will be required in the next part

In [11]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Ontario are 43.653963, -79.387207.


Using Folium to Map

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'], can_postal_cd_df['PostalCode'], can_postal_cd_df['Borough'], can_postal_cd_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

### PART-2 Ends Here

## PART-3 Explore and Cluster Neighbourhoods in Toronto

#### Step - 1: Filtering out Toronto Boroughs in the complete Data

In [13]:
# Printing all Borough values in the Pandas Dataframe
can_postal_cd_df.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [14]:
toronto_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_all_df = can_postal_cd_df[can_postal_cd_df['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
print("Shape of All Toronto Data dataframe is - ",toronto_all_df.shape)
toronto_all_df.head(40)

Shape of All Toronto Data dataframe is -  (38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676845,-79.295225
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085
5,M4P,Central Toronto,Davisville North,43.712815,-79.388526
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
7,M4S,Central Toronto,Davisville,43.703395,-79.385964
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.690655,-79.383561
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686083,-79.402335


In [15]:
#Mapping only Toronto Data which was filtered earlier, we can see the variance in both maps which had the locations marked
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, post, borough, neigh in zip(toronto_all_df['Latitude'], toronto_all_df['Longitude'], toronto_all_df['PostalCode'], toronto_all_df['Borough'], toronto_all_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

#### Step - 2:Using Foursquare API going forward:

In [16]:
CLIENT_ID = 'PDUBQ0JA2ZYZG1VB00LUWNZDAYNXQABP0EJAWOOHQLGQF02I' # your Foursquare ID
CLIENT_SECRET = 'PKW0KCTBHBBOWCDGWWTAOIJLIKLT0MZV5CHPC4TIAVHG2LDC' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PDUBQ0JA2ZYZG1VB00LUWNZDAYNXQABP0EJAWOOHQLGQF02I
CLIENT_SECRET:PKW0KCTBHBBOWCDGWWTAOIJLIKLT0MZV5CHPC4TIAVHG2LDC


### All recommended places in each Borough in Toronto

In [17]:
radius = 500
LIMIT = 1000

recommends = []
for lat, long, post, borough, neighborhood in zip(toronto_all_df['Latitude'], toronto_all_df['Longitude'], toronto_all_df['PostalCode'], toronto_all_df['Borough'], toronto_all_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    postal_data = requests.get(url).json()["response"]['groups'][0]['items']
    for recommend_post in postal_data:
        recommends.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            recommend_post['venue']['name'], 
            recommend_post['venue']['location']['lat'], 
            recommend_post['venue']['location']['lng'],  
            recommend_post['venue']['categories'][0]['name']))

KeyError: 'groups'

In [ ]:
toronto_recommends = pd.DataFrame(recommends)
toronto_recommends.columns = ['PostalCode', 'Borough', 'Neighborhoods', 'Borough_Lat', 'Borough_Long', 'Venue_Name', 'Venue_Lat', 'Venue_Long', 'Venue_Category']
print("Shape of all the recommendations of all the Postal Codes in Toronto - ", toronto_recommends.shape)
toronto_recommends.head()

In [ ]:
#Removing any duplicates if any
toronto_recommends.drop_duplicates(keep=False, inplace=True)
toronto_recommends.shape

In [ ]:
toronto_recommends.groupby(['PostalCode', 'Borough', 'Neighborhoods'])['Venue_Name'].count()

In [ ]:
print("Number of Unique Venue Categories are: ",len(toronto_recommends['Venue_Category'].unique()))
toronto_recommends['Venue_Category'].unique()

### Analyze venues in each area

In [ ]:
# getting all the values as columns for all areas
toronto_recommends = toronto_recommends.drop(['Borough_Lat','Borough_Long','Venue_Lat','Venue_Long'],axis=1)
toronto_recommends_df = pd.get_dummies(toronto_recommends, columns=['Venue_Category'],prefix = "", prefix_sep = "")
toronto_recommends_df.head(40)

In [ ]:
toronto_venues_freq = toronto_recommends_df.groupby(['PostalCode','Borough','Neighborhoods']).mean().reset_index()
toronto_venues_freq.head(40)

### Top 10 Venues in the each of the Areas

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# Create a new dataframe
all_toronto_venues = pd.DataFrame(columns=columns)
all_toronto_venues['PostalCode'] = toronto_venues_freq['PostalCode']
all_toronto_venues['Borough'] = toronto_venues_freq['Borough']
all_toronto_venues['Neighborhoods'] = toronto_venues_freq['Neighborhoods']
for ind in np.arange(toronto_venues_freq.shape[0]):
    row_categories = toronto_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    all_toronto_venues.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

all_toronto_venues.sort_values(freqColumns, inplace=True)
all_toronto_venues

### Clustering areas
Use KMeans algorigthm, try to cluster the toronto central areas into 5 clusters

In [ ]:
# set number of clusters
kclusters = 5

toronto_venues_freq_clustering = toronto_venues_freq.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_venues_freq_clustering)

#No data for a particular postal code so removed it
toronto_clustered_df = toronto_all_df.drop(toronto_all_df.index[toronto_all_df.PostalCode=='M5N'])
toronto_clustered_df['Cluster'] = kmeans.labels_

toronto_clustered_df = toronto_clustered_df.join(all_toronto_venues.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
toronto_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
toronto_clustered_df

### Mapping Clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_clustered_df['Latitude'], toronto_clustered_df['Longitude'], toronto_clustered_df['PostalCode'], toronto_clustered_df['Borough'], toronto_clustered_df['Neighborhood'], toronto_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5 Colours - Clusters plotted above

### Assignment is prepared by 
<i><b>Thet Paing Soe</b></i>